In [2]:
import numpy as np
import h5py
from glob import glob
from tqdm import tqdm

# !pip3 uninstall -q open3d-python
import open3d as o3d
# from open3d.open3d.geometry import estimate_normals

ImportError: /lib64/libstdc++.so.6: version `CXXABI_1.3.8' not found (required by /trinity/home/a.matveev/.local/lib/python3.6/site-packages/open3d/open3d.cpython-36m-x86_64-linux-gnu.so)

In [2]:
addrs = np.concatenate([glob('../data/images_real/test/*.hdf5')])

In [3]:
addrs

array(['../data/images_real/test/22top_folder_images.hdf5',
       '../data/images_real/test/5top_images.hdf5',
       '../data/images_real/test/43top_folder_images.hdf5',
       '../data/images_real/test/6side_folder_images.hdf5',
       '../data/images_real/test/89side_folder_images.hdf5',
       '../data/images_real/test/5side_images.hdf5',
       '../data/images_real/test/9top_folder_images.hdf5',
       '../data/images_real/test/40side_folder_images.hdf5',
       '../data/images_real/test/1top_folder_images.hdf5',
       '../data/images_real/test/10top_folder_images.hdf5',
       '../data/images_real/test/63top_folder_images.hdf5',
       '../data/images_real/test/11top_images.hdf5',
       '../data/images_real/test/16side_folder_images.hdf5',
       '../data/images_real/test/77top_folder_images.hdf5',
       '../data/images_real/test/35side_folder_images.hdf5',
       '../data/images_real/test/10side_folder_images.hdf5',
       '../data/images_real/test/43side_folder_images.hdf5'

In [4]:
def image_to_points(image):
    image_height, image_width = image.shape
    resolution_3d = 0.02
    screen_aspect_ratio = 1

    rays_screen_coords = np.mgrid[0:image_height, 0:image_width].reshape(
            2, image_height * image_width).T

    rays_origins = (rays_screen_coords / np.array([[image_height, image_width]]))   # [h, w, 2], in [0, 1]
    factor = image_height / 2 * resolution_3d
    rays_origins[:, 0] = (-2 * rays_origins[:, 0] + 1) * factor  # to [-1, 1] + aspect transform
    rays_origins[:, 1] = (-2 * rays_origins[:, 1] + 1) * factor * screen_aspect_ratio
    rays_origins = np.concatenate([
        rays_origins,
        np.zeros_like(rays_origins[:, [0]])
    ], axis=1)

    i = np.where(image.ravel() != 0)[0]
    points = np.zeros((len(i), 3))
    points[:, 0] = rays_origins[i, 0]
    points[:, 1] = rays_origins[i, 1]
    points[:, 2] = image.ravel()[i]
    return points

In [4]:
for addr in tqdm(addrs):
    with h5py.File(addr, 'r') as f:
        data = f['points'][:]

    n_10 = []
    n_100 = []
    for d in data:
#         print('0')
#         p = image_to_points(d)
#         print('1')
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(d.reshape(-1,3))
#         pcd.points = o3d.utility.Vector3dVector(p)
        pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(10), fast_normal_computation=False)
        n_10.append(np.asarray(pcd.normals).flatten())
        pcd.clear()
#         print('2')
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(d.reshape(-1,3))
#         pcd.points = o3d.utility.Vector3dVector(p)
        pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(100), fast_normal_computation=False)
        n_100.append(np.asarray(pcd.normals).flatten())
        pcd.clear()

    with h5py.File(addr, 'a') as f:
        dataset_10 = f.create_dataset("normals_estimation_10", shape=(len(data), ), dtype=h5py.special_dtype(vlen=np.float64))
        dataset_100 = f.create_dataset("normals_estimation_100", shape=(len(data), ), dtype=h5py.special_dtype(vlen=np.float64))
        for i in range(len(n_10)):
            dataset_10[i] = n_10[i]
            dataset_100[i] = n_100[i]

100%|██████████| 65/65 [13:56<00:00, 12.86s/it]


In [6]:
addrs = np.concatenate([glob('../data/images/high_res_0.005/*.hdf5'), 
                        glob('../data/images/high_res_0.02/*.hdf5'), 
                        glob('../data/images/high_res_0.08/*.hdf5')])

In [9]:
for addr in tqdm(addrs):
    with h5py.File(addr, 'r') as f:
        print(list(f.keys()))
        data = f['points'][:]

    n_10 = []
    n_100 = []
    for d in data:
#         print('0')
        p = image_to_points(d)
#         print('1')
        pcd = o3d.geometry.PointCloud()
#         pcd.points = o3d.utility.Vector3dVector(d.reshape(-1,3))
        pcd.points = o3d.utility.Vector3dVector(p)
        pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(10), fast_normal_computation=False)
        n_10.append(np.asarray(pcd.normals).flatten())
        pcd.clear()
#         print('2')
        pcd = o3d.geometry.PointCloud()
#         pcd.points = o3d.utility.Vector3dVector(d.reshape(-1,3))
        pcd.points = o3d.utility.Vector3dVector(p)
        pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(100), fast_normal_computation=False)
        n_100.append(np.asarray(pcd.normals).flatten())
        pcd.clear()

    with h5py.File(addr, 'a') as f:
        f.create_dataset('image', data=data)
        dataset_10 = f.create_dataset("normals_estimation_10", shape=(len(data), ), dtype=h5py.special_dtype(vlen=np.float64))
        dataset_100 = f.create_dataset("normals_estimation_100", shape=(len(data), ), dtype=h5py.special_dtype(vlen=np.float64))
        for i in range(len(n_10)):
            dataset_10[i] = n_10[i]
            dataset_100[i] = n_100[i]

  0%|          | 0/29 [00:00<?, ?it/s]

['directions', 'distances', 'extrinsics', 'faces', 'has_sharp', 'intrinsics', 'item_id', 'num_sharp_curves', 'num_surfaces', 'obj_alignment', 'obj_scale', 'orig_face_indexes', 'orig_vert_indices', 'points', 'points_alignment']


  3%|▎         | 1/29 [00:02<01:20,  2.88s/it]

['directions', 'distances', 'extrinsics', 'faces', 'has_sharp', 'intrinsics', 'item_id', 'num_sharp_curves', 'num_surfaces', 'obj_alignment', 'obj_scale', 'orig_face_indexes', 'orig_vert_indices', 'points', 'points_alignment']


  7%|▋         | 2/29 [00:06<01:20,  2.99s/it]

...

 93%|█████████▎| 27/29 [01:19<00:05,  2.94s/it]

['directions', 'distances', 'extrinsics', 'faces', 'has_sharp', 'intrinsics', 'item_id', 'num_sharp_curves', 'num_surfaces', 'obj_alignment', 'obj_scale', 'orig_face_indexes', 'orig_vert_indices', 'points', 'points_alignment']


 97%|█████████▋| 28/29 [01:21<00:02,  2.84s/it]

['directions', 'distances', 'extrinsics', 'faces', 'has_sharp', 'intrinsics', 'item_id', 'num_sharp_curves', 'num_surfaces', 'obj_alignment', 'obj_scale', 'orig_face_indexes', 'orig_vert_indices', 'points', 'points_alignment']


100%|██████████| 29/29 [01:24<00:00,  2.93s/it]


In [8]:
def image_to_points(image):
    image_height, image_width = image.shape
    resolution_3d = 0.02
    screen_aspect_ratio = 1

    rays_screen_coords = np.mgrid[0:image_height, 0:image_width].reshape(
            2, image_height * image_width).T.astype(np.float)

    rays_origins = (rays_screen_coords / np.array([[image_height, image_width]]))   # [h, w, 2], in [0, 1]
    
    factor = image_height / 2 * resolution_3d
    rays_origins[:, 0] = (-2 * rays_origins[:, 0] + 1) * factor  # to [-1, 1] + aspect transform
    rays_origins[:, 1] = (-2 * rays_origins[:, 1] + 1) * factor * screen_aspect_ratio
    
    rays_origins = np.concatenate([
        rays_origins,
        np.zeros_like(rays_origins[:, [0]])
    ], axis=1)

    i = np.where(image.ravel() != 0)[0]
    points = np.zeros((len(i), 3))
    points[:, 0] = rays_origins[i, 0]
    points[:, 1] = rays_origins[i, 1]
    points[:, 2] = image.ravel()[i]
    return points, i

In [10]:
with h5py.File('../data/images_real/test/10top_folder_images.hdf5', 'r') as f:
        data = list(f['image'])
        points = []
        nonzero = []
        for i, data_i in tqdm(enumerate(data)):
            tmp, tmp_nz = image_to_points(data_i)
            points.append(tmp)
            nonzero.append(tmp_nz)
        data = points
        if True:
            norms = list(f['normals_estimation_100'])
            normals = []
            for i in range(len(data)):
                normals.append(np.concatenate([data[i], norms[i].reshape(-1,3)], axis = -1))
#             out = []
#             for i in range(len(data)):
#                 out.append(np.concatenate([data[i], normals[i]], axis = -1))
#             data = np.concatenate([data, normals], axis = -1)
#             print(data.shape)
            data = normals

12it [00:02,  5.88it/s]


IndexError: index 1725308 is out of bounds for axis 0 with size 10512

In [16]:
with h5py.File('../data/images_real/test/10top_folder_images.hdf5', 'r') as f:
    n = f['normals_estimation_100'][:]

In [22]:
norms[i].reshape(-1,3).shape

(3504, 3)

In [7]:
for addr in tqdm(addrs):
    with h5py.File(addr, 'r') as f:
        print(list(f.keys()))
        data = f['points'][:]

  0%|          | 0/29 [00:00<?, ?it/s]

['directions', 'distances', 'extrinsics', 'faces', 'has_sharp', 'intrinsics', 'item_id', 'num_sharp_curves', 'num_surfaces', 'obj_alignment', 'obj_scale', 'orig_face_indexes', 'orig_vert_indices', 'points', 'points_alignment']


  3%|▎         | 1/29 [00:00<00:14,  1.91it/s]

['directions', 'distances', 'extrinsics', 'faces', 'has_sharp', 'intrinsics', 'item_id', 'num_sharp_curves', 'num_surfaces', 'obj_alignment', 'obj_scale', 'orig_face_indexes', 'orig_vert_indices', 'points', 'points_alignment']


  7%|▋         | 2/29 [00:01<00:13,  1.95it/s]

...

['directions', 'distances', 'extrinsics', 'faces', 'has_sharp', 'intrinsics', 'item_id', 'num_sharp_curves', 'num_surfaces', 'obj_alignment', 'obj_scale', 'orig_face_indexes', 'orig_vert_indices', 'points', 'points_alignment']


 59%|█████▊    | 17/29 [00:08<00:05,  2.07it/s]

['directions', 'distances', 'extrinsics', 'faces', 'has_sharp', 'intrinsics', 'item_id', 'num_sharp_curves', 'num_surfaces', 'obj_alignment', 'obj_scale', 'orig_face_indexes', 'orig_vert_indices', 'points', 'points_alignment']


 59%|█████▊    | 17/29 [00:08<00:06,  1.94it/s]


KeyboardInterrupt: 

In [8]:
data.shape

(12, 1536, 2048)

In [8]:
p.shape

(125905, 3)

In [9]:
from open3d.open3d.geometry import estimate_normals

In [22]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(d.reshape(-1,3))
estimate_normals(pcd, o3d.geometry.KDTreeSearchParamKNN(10))

True

In [23]:
pcd??

In [2]:
with h5py.File('train_0.hdf5', 'r') as f:
    data = f['points'][:]

In [60]:
d.mean(axis=0)

array([ 0.94612691,  7.44326331, -1.69337792])

In [41]:
addrs

['train_1.hdf5',
 'val_1.hdf5',
 'train_8.hdf5',
 'train_0.hdf5',
 'train_2.hdf5',
 'test_1.hdf5',
 'train_3.hdf5',
 'train_5.hdf5',
 'val_0.hdf5',
 'train_9.hdf5',
 'train_4.hdf5',
 'test_0.hdf5',
 'train_10.hdf5',
 'train_6.hdf5',
 'train_7.hdf5']

In [4]:
# addrs = glob('*.hdf5')
for i, addr in enumerate(addrs):
    print(i)
    with h5py.File(addr, 'r') as f:
        print(f['normals_estimation_10'][:].shape)
        print(f['normals_estimation_100'][:].shape)

0
(16384, 4096, 3)
(16384, 4096, 3)
1
(14752, 4096, 3)
(14752, 4096, 3)
2
(16384, 4096, 3)
(16384, 4096, 3)
3
(14435, 4096, 3)
(14435, 4096, 3)
4
(16384, 4096, 3)
(16384, 4096, 3)
5
(15585, 4096, 3)
(15585, 4096, 3)
6
(16384, 4096, 3)
(16384, 4096, 3)
7
(15320, 4096, 3)
(15320, 4096, 3)
8
(16384, 4096, 3)
(16384, 4096, 3)
9
(15574, 4096, 3)
(15574, 4096, 3)
10
(16384, 4096, 3)
(16384, 4096, 3)
11
(14433, 4096, 3)


KeyError: "Unable to open object (object 'normals_estimation_100' doesn't exist)"

'tmp/data/high/0.005/val_1.hdf5'

In [6]:
addrs = np.sort(glob('data/full/*/med_res/*/*.hdf5'))

In [7]:
addrs

array(['data/full/points/med_res/0.0/abc_0050_00500082_4cb4bf14428fe3832dd7ed78_000.hdf5',
       'data/full/points/med_res/0.0/abc_0050_00500166_5894bbd701b2bb0fc88a6978_007.hdf5',
       'data/full/points/med_res/0.0/abc_0050_00500348_fae0ecd8b3dc068d39f0d09c_000.hdf5',
       'data/full/points/med_res/0.0/abc_0050_00500683_511f7debb63f164003339dec_000.hdf5',
       'data/full/points/med_res/0.0/abc_0050_00501216_5894ff84fca3da0f6b67497b_012.hdf5',
       'data/full/points/med_res/0.0/abc_0050_00505574_5897be3fa417220f72333795_000.hdf5',
       'data/full/points/med_res/0.0/abc_0050_00506880_f982cf821a90f24a03bd661f_000.hdf5',
       'data/full/points/med_res/0.0/abc_0050_00507625_b39680dc8dc650ec01956bbe_011.hdf5'],
      dtype='<U80')

In [8]:
for addr in addrs:
    with h5py.File(addr, 'r') as f:
        data = f['points'][:]

    n_10 = []
    n_100 = []
    for d in tqdm(data):
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(d.reshape(-1,3))
        pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(10), fast_normal_computation=False)
        n_10.append(np.asarray(pcd.normals).flatten())
        pcd.clear()
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(d.reshape(-1,3))
        pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(100), fast_normal_computation=False)
        n_100.append(np.asarray(pcd.normals).flatten())
        pcd.clear()

    with h5py.File(addr, 'a') as f:
        dataset_10 = f.create_dataset("normals_estimation_10", shape=(len(data), ), dtype=h5py.special_dtype(vlen=np.float64))
        dataset_100 = f.create_dataset("normals_estimation_100", shape=(len(data), ), dtype=h5py.special_dtype(vlen=np.float64))
        for i in range(len(n_10)):
            dataset_10[i] = n_10[i]
            dataset_100[i] = n_100[i]
#     break

100%|██████████| 347/347 [00:25<00:00, 13.68it/s]


In [45]:
# with h5py.File(addr, 'r') as f:
#     data = f['points'][:]

# n_10 = []
# n_100 = []
# for d in tqdm(data):
#     pcd = o3d.geometry.PointCloud()
#     pcd.points = o3d.utility.Vector3dVector(d.reshape(-1,3))
#     pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(10), fast_normal_computation=False)
#     n_10.append(np.asarray(pcd.normals).flatten())
#     pcd.clear()
#     pcd = o3d.geometry.PointCloud()
#     pcd.points = o3d.utility.Vector3dVector(d.reshape(-1,3))
#     pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(100), fast_normal_computation=False)
#     n_100.append(np.asarray(pcd.normals).flatten())
#     pcd.clear()

with h5py.File(addrs[1], 'r') as f:
    print(f['image'][:].shape)
#     del f["normals_estimation_10"]
#     del f["normals_estimation_100"]
#     dataset_10 = f.create_dataset("normals_estimation_10", shape=(len(data), ), dtype=h5py.special_dtype(vlen=np.float64))
#     dataset_100 = f.create_dataset("normals_estimation_100", shape=(len(data), ), dtype=h5py.special_dtype(vlen=np.float64))
#     for i in range(len(n_10)):
#         dataset_10[i] = n_10[i]
#         dataset_100[i] = n_100[i]

(18, 2024, 2024)


In [39]:
17430/3

5810.0

In [22]:
with h5py.File('test.hdf5', 'w') as f:
    dataset = f.create_dataset('test', shape=(5, ), dtype=h5py.special_dtype(vlen=np.float64))
    for i, item in enumerate(range(5)):
        dataset[i] = np.arange(i+1)


In [2]:
addrs = glob('data/str/*.hdf5')

In [3]:
addrs

['data/str/Castle_315.hdf5',
 'data/str/hook_345.hdf5',
 'data/str/smallest_40.hdf5',
 'data/str/Castle_45.hdf5',
 'data/str/small_40.hdf5',
 'data/str/cube_small_345.hdf5',
 'data/str/bat_320.hdf5',
 'data/str/Big_40.hdf5',
 'data/str/Cross_345.hdf5',
 'data/str/small_345.hdf5',
 'data/str/Detail_315.hdf5',
 'data/str/Big_345.hdf5']

In [5]:
for addr in addrs:
    with h5py.File(addr, 'r') as f:
        data = f['points'][:]

    n_10 = []
    n_100 = []
    for d in tqdm(data):
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(d)
        pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(10), fast_normal_computation=False)
        n_10.append(np.asarray(pcd.normals))
        pcd.clear()
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(d)
        pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(100), fast_normal_computation=False)
        n_100.append(np.asarray(pcd.normals))
        pcd.clear()

    with h5py.File(addr, 'a') as f:
        f.create_dataset("normals_estimation_10", data=np.array(n_10))
        f.create_dataset("normals_estimation_100", data=np.array(n_100))

100%|██████████| 115/115 [00:06<00:00, 17.06it/s]


In [13]:
for i in range(2):
    with h5py.File('test_'+str(i)+'.hdf5', 'r') as f:
        data = f['points'][:]

    n_10 = []
    n_100 = []
    for d in tqdm(data):
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(d)
        pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(10), fast_normal_computation=False)
        n_10.append(np.asarray(pcd.normals))
        pcd.clear()
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(d)
        pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(100), fast_normal_computation=False)
        n_100.append(np.asarray(pcd.normals))
        pcd.clear()

    with h5py.File('test_'+str(i)+'.hdf5', 'a') as f:
        f.create_dataset("normals_estimation_10", data=np.array(n_10))
        f.create_dataset("normals_estimation_100", data=np.array(n_100))

100%|██████████| 15601/15601 [02:10<00:00, 119.81it/s]


In [4]:
with h5py.File('train_0.hdf5', 'a') as f:
    f.create_dataset("normals_estimation_10", data=np.array(n_10))
    f.create_dataset("normals_estimation_100", data=np.array(n_100))

In [111]:
np.save('n_10_0', n_10)

Object `open3d.open3d` not found.


In [70]:
pcd.clear()

In [71]:
pcd

geometry::PointCloud with 0 points.

In [102]:
for i in n[0]:
    print(i)

[ 0.99566791 -0.09298066  0.        ]
[1. 0. 0.]
[-1.  0.  0.]
[-1.  0.  0.]
[-0.99879776  0.04902075  0.        ]
[-0.85749293 -0.51449576  0.        ]
[-1.  0.  0.]
[-0.98879909  0.14925269  0.        ]
[0.94337248 0.33173538 0.        ]
[-0.92847669 -0.37139068  0.        ]
[0.74740932 0.66436384 0.        ]
[ 0.8479983  -0.52999894  0.        ]
[-0.9883717   0.15205718  0.        ]
[-0.9966461   0.08183246  0.        ]
[ 0.70710678 -0.70710678  0.        ]
[-1.  0.  0.]
[-1.  0.  0.]
[-0.99949197 -0.03187156  0.        ]
[-0.97230237 -0.23372653  0.        ]
[-0.99450545  0.10468478  0.        ]
[1. 0. 0.]
[-0.87415728 -0.48564293  0.        ]
[1. 0. 0.]
[1. 0. 0.]
[-1.  0.  0.]
[-1.  0.  0.]
[ 0.95152728 -0.30756437  0.        ]
[ 0.95577901 -0.29408585  0.        ]
[0.83205029 0.5547002  0.        ]
[1. 0. 0.]
[1. 0. 0.]
[-0.97874026 -0.20510366  0.        ]
[-1.  0.  0.]
[1. 0. 0.]
[-0.90795938  0.41905818  0.        ]
[0.95103407 0.30908607 0.        ]
[-0.99478144 -0.10202887 

In [16]:
import numpy as np

from IPython.display import HTML
import trimesh
import k3d

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm

def get_colors(value, cmap):
    hex_colors = []
    norm = mpl.colors.Normalize(vmin=value.min(), vmax=value.max())
    cmap = cmap
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    for i in value:
#         hex_colors.append(m.to_rgba(i))
        hex_colors.append(int(mpl.colors.to_hex(m.to_rgba(i)).replace('#','0x'),16))
    hex_colors = np.array(hex_colors, 'uint32')
    return(hex_colors)

def show_points(points, colors=[], normals=None, point_size=0.1, line_width=0.00001):
    plot = k3d.plot(grid_visible=False, axes_helper=0)
    if normals is not None:
        normal_vectors = k3d.vectors(points, normals, line_width=line_width, use_head=False)
        plot += normal_vectors
    point_cloud = k3d.points(points, colors=colors, point_size=point_size, shader='flat')
    plot += point_cloud
    plot.display()
    return plot

In [17]:
import numpy as np

In [19]:
p = show_points(np.asarray(pcd.points), normals=np.asarray(pcd.normals)/5)
with open("n.html", 'w') as f:
    f.write(p.get_snapshot())

Output()

In [49]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(data[14])
estimate_normals(pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

True

In [68]:
np.asarray(pcd.normals)

array([[-0.81373347,  0.58123819,  0.        ],
       [-0.97618706, -0.21693046,  0.        ],
       [-1.        ,  0.        ,  0.        ],
       ...,
       [-1.        ,  0.        ,  0.        ],
       [-0.99999504, -0.00315051,  0.        ],
       [ 1.        ,  0.        ,  0.        ]])

In [43]:
np.where(l.sum(-1) < 4096)

(array([   14,    15,    21, ..., 16377, 16380, 16382]),)

In [2]:
def image_to_points(image):
    image_height, image_width = image.shape
    resolution_3d = 0.02
    screen_aspect_ratio = 1

    rays_screen_coords = np.mgrid[0:image_height, 0:image_width].reshape(
            2, image_height * image_width).T

    rays_origins = (rays_screen_coords / np.array([[image_height, image_width]]))   # [h, w, 2], in [0, 1]
    factor = image_height / 2 * resolution_3d
    rays_origins[:, 0] = (-2 * rays_origins[:, 0] + 1) * factor  # to [-1, 1] + aspect transform
    rays_origins[:, 1] = (-2 * rays_origins[:, 1] + 1) * factor * screen_aspect_ratio
    rays_origins = np.concatenate([
        rays_origins,
        np.zeros_like(rays_origins[:, [0]])
    ], axis=1)

    i = np.where(image.ravel() != 0)[0]
    points = np.zeros((image_height * image_width, 3))
    points[:, 0] = rays_origins[:, 0]
    points[:, 1] = rays_origins[:, 1]
    points[:, 2] = image.ravel()[:]
    return points

In [9]:
addrs = glob('/trinity/home/a.matveev/data/full/images/low_res/*/*.hdf5')

In [10]:
addrs

['/trinity/home/a.matveev/data/full/images/low_res/0.0/abc_0050_00500348_fae0ecd8b3dc068d39f0d09c_000.hdf5',
 '/trinity/home/a.matveev/data/full/images/low_res/0.0/abc_0050_00507625_b39680dc8dc650ec01956bbe_011.hdf5',
 '/trinity/home/a.matveev/data/full/images/low_res/0.0/abc_0050_00500166_5894bbd701b2bb0fc88a6978_007.hdf5',
 '/trinity/home/a.matveev/data/full/images/low_res/0.0/abc_0050_00505574_5897be3fa417220f72333795_000.hdf5',
 '/trinity/home/a.matveev/data/full/images/low_res/0.0/abc_0050_00500082_4cb4bf14428fe3832dd7ed78_000.hdf5',
 '/trinity/home/a.matveev/data/full/images/low_res/0.0/abc_0050_00506880_f982cf821a90f24a03bd661f_000.hdf5',
 '/trinity/home/a.matveev/data/full/images/low_res/0.0/abc_0050_00501216_5894ff84fca3da0f6b67497b_012.hdf5',
 '/trinity/home/a.matveev/data/full/images/low_res/0.0/abc_0050_00500149_54930d6f7740b03347d89a56_000.hdf5',
 '/trinity/home/a.matveev/data/full/images/low_res/0.0/abc_0050_00500683_511f7debb63f164003339dec_000.hdf5']

In [6]:
for addr in addrs:
    with h5py.File(addr, 'r') as f:
        print(len(f['image'][:]))

18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18


In [11]:
for addr in addrs:
    with h5py.File(addr, 'r') as f:
        data = f['image'][:]

    n_10 = []
    n_100 = []
    for d in tqdm(data):
        p = image_to_points(d)
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(p)
        pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(10), fast_normal_computation=False)
        n_10.append(np.asarray(pcd.normals))
        pcd.clear()
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(p)
        pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(100), fast_normal_computation=False)
        n_100.append(np.asarray(pcd.normals))
        pcd.clear()

    with h5py.File(addr, 'a') as f:
        f.create_dataset("normals_estimation_10", data=np.array(n_10))
        f.create_dataset("normals_estimation_100", data=np.array(n_100))

100%|██████████| 18/18 [03:02<00:00, 10.16s/it]


In [ ]:
?

In [7]:
np.array(n_10)

array([array([[-0.94485802, -0.32046094, -0.06743969],
       [-0.94485802, -0.32046094, -0.06743969],
       [-0.94485802, -0.32046094, -0.06743969],
       ...,
       [-0.94485802, -0.32046094, -0.06743969],
       [-0.94485802, -0.32046094, -0.06743969],
       [-0.94485802, -0.32046094, -0.06743969]]),
       array([[-0.93838035, -0.33934748, -0.0654645 ],
       [ 0.97143352,  0.19822405,  0.13047656],
       [ 0.08343599,  0.95594234,  0.28144748],
       [-0.44702495,  0.87335933, -0.19342228],
       [ 0.94548029,  0.31222747,  0.09263388],
       [ 0.98353982, -0.07082552,  0.1662323 ],
       [ 0.94548029,  0.31222747,  0.09263388],
       [ 0.12935837,  0.96908748, -0.21008541],
       [-0.42272421,  0.57408184,  0.70123768],
       [-0.97261786, -0.16640865, -0.16224262],
       [ 0.96496839,  0.11037295,  0.23802061],
       [ 0.96496839,  0.11037295,  0.23802061],
       [-0.98997116,  0.08321146, -0.11416198],
       [ 0.98539645,  0.13494086,  0.10384991],
       [ 0.4

In [13]:
for res in ['med','low']:

    for i in range(0,2):
        with h5py.File('tmp/data/'+res+'/points/val_'+str(i)+'.hdf5', 'r') as f:
            data = f['points'][:]

        n_10 = []
        n_100 = []
        for p in tqdm(data):
#             p = image_to_points(d)
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(p)
            pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(10), fast_normal_computation=False)
            n_10.append(np.asarray(pcd.normals))
            pcd.clear()
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(p)
            pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(100), fast_normal_computation=False)
            n_100.append(np.asarray(pcd.normals))
            pcd.clear()

        with h5py.File('tmp/data/'+res+'/points/val_'+str(i)+'.hdf5', 'a') as f:
            f.create_dataset("normals_estimation_10", data=np.array(n_10))
            f.create_dataset("normals_estimation_100", data=np.array(n_100))

100%|██████████| 16384/16384 [02:19<00:00, 117.09it/s]


In [19]:
for res in ['med']:

    for i in range(1,2):
        with h5py.File('tmp/data/'+res+'/points/test_'+str(i)+'.hdf5', 'r') as f:
            data = f['points'][:]

        n_10 = []
        n_100 = []
        for p in tqdm(data):
#             p = image_to_points(d)
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(p)
            pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(10), fast_normal_computation=False)
            n_10.append(np.asarray(pcd.normals))
            pcd.clear()
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(p)
            pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(100), fast_normal_computation=False)
            n_100.append(np.asarray(pcd.normals))
            pcd.clear()

        with h5py.File('tmp/data/'+res+'/points/test_'+str(i)+'.hdf5', 'a') as f:
            f.create_dataset("normals_estimation_10", data=np.array(n_10))
            f.create_dataset("normals_estimation_100", data=np.array(n_100))

100%|██████████| 16384/16384 [02:16<00:00, 119.76it/s]


In [10]:
with h5py.File('tmp/data/high/0.02/test_0.hdf5', 'a') as f:
    print(list(f.keys()))

['camera_pose', 'directions', 'distances', 'has_sharp', 'has_smell_bad_face_sampling', 'has_smell_coarse_surfaces_by_angles', 'has_smell_coarse_surfaces_by_num_faces', 'has_smell_depth_discontinuity', 'has_smell_deviating_resolution', 'has_smell_mesh_self_intersections', 'has_smell_mismatching_surface_annotation', 'has_smell_raycasting_background', 'has_smell_sharpness_discontinuities', 'image', 'item_id', 'mesh_scale', 'normals', 'normals_estimation_10', 'normals_estimation_100', 'num_sharp_curves', 'num_surfaces', 'orig_face_indexes', 'orig_vert_indices']


In [11]:
for i in range(0,2):
    with h5py.File('tmp/test_1/train_'+str(i)+'.hdf5', 'r') as f:
        data = f['image'][:]

    n_10 = []
    n_100 = []
    for d in tqdm(data):
        p = image_to_points(d)
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(p)
        pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(10), fast_normal_computation=False)
        n_10.append(np.asarray(pcd.normals))
        pcd.clear()
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(p)
        pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(100), fast_normal_computation=False)
        n_100.append(np.asarray(pcd.normals))
        pcd.clear()

    with h5py.File('tmp/test_1/train_'+str(i)+'.hdf5', 'a') as f:
        f.create_dataset("normals_estimation_10", data=np.array(n_10))
        f.create_dataset("normals_estimation_100", data=np.array(n_100))

100%|██████████| 16384/16384 [02:51<00:00, 95.62it/s] 


In [13]:
with h5py.File('train_0.hdf5', 'r') as f:
    d = f['image'][:]
    n_c = f['num_sharp_curves'][:]

In [14]:
s = n_c > 0

In [26]:
p = image_to_points(d[10])

In [31]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(p)
pcd.estimate_normals(o3d.geometry.KDTreeSearchParamKNN(10), fast_normal_computation=False)

True

In [32]:
show_points(p, normals=np.asarray(pcd.normals)/5)

Output()

Plot(antialias=3, axes=['x', 'y', 'z'], background_color=16777215, camera=[4.5, 4.5, 4.5, 0.0, 0.0, 0.0, 1.0, …

In [52]:

#     f.close()

In [7]:
x = np.linspace(-3, 3, 201)
mesh_x, mesh_y = np.meshgrid(x, x)
z = np.sinc((np.power(mesh_x, 2) + np.power(mesh_y, 2)))
z_norm = (z - z.min()) / (z.max() - z.min())
xyz = np.zeros((np.size(mesh_x), 3))
xyz[:, 0] = np.reshape(mesh_x, -1)
xyz[:, 1] = np.reshape(mesh_y, -1)
xyz[:, 2] = np.reshape(z_norm, -1)
print('xyz')
print(xyz)

xyz
[[-3.         -3.          0.17846415]
 [-2.97       -3.          0.17063652]
 [-2.94       -3.          0.16512557]
 ...
 [ 2.94        3.          0.16512557]
 [ 2.97        3.          0.17063652]
 [ 3.          3.          0.17846415]]


In [8]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)

In [5]:
np.asarray(pcd.points)


array([[5.36196354, 7.68534978, 0.        ],
       [5.3435505 , 7.68005602, 0.        ],
       [5.38020498, 7.6978317 , 0.        ],
       ...,
       [6.24824672, 7.50914827, 0.        ],
       [4.45498521, 7.64581376, 0.        ],
       [6.25386114, 7.83987371, 0.        ]])

In [9]:
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

AttributeError: 'open3d.open3d.geometry.PointCloud' object has no attribute 'estimate_normals'

In [10]:
from open3d.open3d.geometry import estimate_normals

In [112]:
estimate_normals??

In [25]:
addrs = np.sort(glob('voronoi/v2/high_res/0.0/*val*.hdf5'))
addrs[0][26:]

'val_0.hdf5'

In [30]:

with h5py.File('voronoi/v2/high_res/0.0/v_test_1.hdf5', 'r') as f:
    v = f['distances'][:]
with h5py.File('data/v2/files/high_res/0.0/test/train_1.hdf5', 'a') as f:
    f.create_dataset("voronoi", data=v)

In [24]:
with h5py.File('data/v2/files/high_res/0.0/train/train_1.hdf5', 'r') as f:
    print(list(f.keys()))

['directions', 'distances', 'has_sharp', 'has_smell_bad_face_sampling', 'has_smell_coarse_surfaces_by_angles', 'has_smell_coarse_surfaces_by_num_faces', 'has_smell_deviating_resolution', 'has_smell_mismatching_surface_annotation', 'has_smell_sharpness_discontinuities', 'item_id', 'normals', 'num_sharp_curves', 'num_surfaces', 'orig_face_indexes', 'orig_vert_indices', 'points', 'voronoi']


In [31]:
with h5py.File('data/v2/files/high_res/0.0/val/train_1.hdf5', 'r') as f:
    data = f['points'][:]
    dist = f['distances'][:]
    label = f['voronoi'][:]
# with h5py.File(addr, 'r') as f:
    

In [5]:
addrs = np.sort(glob('data/v2/files/high_res/0.0/train/xyz_splitted/train_1_*.xyz'))
len(addrs)

16384

In [2]:
# for i in 
addrs = np.sort(glob('data/v2/files/high_res/0.0/val/xyz_splitted/train_0_*.xyz'))
ns = []
for i in tqdm(range(len(addrs))):
    ns.append(np.loadtxt('data/v2/files/high_res/0.0/val/xyz_splitted/train_0_points_'+str(i)+'.xyz')[:,3:])
np.save('data/v2/files/high_res/0.0/val/10_train_0', ns)

100%|██████████| 16384/16384 [12:05<00:00, 22.57it/s]


In [3]:
addrs = np.sort(glob('data/v2/files/high_res/0.0/val/xyz_splitted/train_1_*.xyz'))
ns = []
for i in tqdm(range(len(addrs))):
    ns.append(np.loadtxt('data/v2/files/high_res/0.0/val/xyz_splitted/train_1_points_'+str(i)+'.xyz')[:,3:])
np.save('data/v2/files/high_res/0.0/val/10_train_1', ns)

100%|██████████| 15412/15412 [13:08<00:00, 19.55it/s]


In [4]:
addrs = np.sort(glob('data/v2/files/high_res/0.0/test/xyz_splitted/train_0_*.xyz'))
ns = []
for i in tqdm(range(len(addrs))):
    ns.append(np.loadtxt('data/v2/files/high_res/0.0/test/xyz_splitted/train_0_points_'+str(i)+'.xyz')[:,3:])
np.save('data/v2/files/high_res/0.0/test/10_train_0', ns)

100%|██████████| 16384/16384 [14:13<00:00, 19.20it/s]


In [5]:
addrs = np.sort(glob('data/v2/files/high_res/0.0/test/xyz_splitted/train_1_*.xyz'))
ns = []
for i in tqdm(range(len(addrs))):
    ns.append(np.loadtxt('data/v2/files/high_res/0.0/test/xyz_splitted/train_1_points_'+str(i)+'.xyz')[:,3:])
np.save('data/v2/files/high_res/0.0/test/10_train_1', ns)

100%|██████████| 15601/15601 [13:24<00:00, 19.38it/s]


In [2]:
# for i in 
addrs = np.sort(glob('data/v2/files/high_res/0.0/val/xyz_splitted_100/train_0_*.xyz'))
ns = []
for i in tqdm(range(len(addrs))):
    ns.append(np.loadtxt('data/v2/files/high_res/0.0/val/xyz_splitted_100/train_0_points_'+str(i)+'.xyz')[:,3:])
np.save('data/v2/files/high_res/0.0/val/100_train_0', ns)

100%|██████████| 16384/16384 [10:27<00:00, 26.12it/s]


In [3]:
addrs = np.sort(glob('data/v2/files/high_res/0.0/val/xyz_splitted_100/train_1_*.xyz'))
ns = []
for i in tqdm(range(len(addrs))):
    ns.append(np.loadtxt('data/v2/files/high_res/0.0/val/xyz_splitted_100/train_1_points_'+str(i)+'.xyz')[:,3:])
np.save('data/v2/files/high_res/0.0/val/100_train_1', ns)

100%|██████████| 15412/15412 [09:42<00:00, 26.48it/s]


In [4]:
addrs = np.sort(glob('data/v2/files/high_res/0.0/test/xyz_splitted_100/train_0_*.xyz'))
ns = []
for i in tqdm(range(len(addrs))):
    ns.append(np.loadtxt('data/v2/files/high_res/0.0/test/xyz_splitted_100/train_0_points_'+str(i)+'.xyz')[:,3:])
np.save('data/v2/files/high_res/0.0/test/100_train_0', ns)

100%|██████████| 16384/16384 [09:33<00:00, 28.58it/s]


In [5]:
addrs = np.sort(glob('data/v2/files/high_res/0.0/test/xyz_splitted_100/train_1_*.xyz'))
ns = []
for i in tqdm(range(len(addrs))):
    ns.append(np.loadtxt('data/v2/files/high_res/0.0/test/xyz_splitted_100/train_1_points_'+str(i)+'.xyz')[:,3:])
np.save('data/v2/files/high_res/0.0/test/100_train_1', ns)

100%|██████████| 15601/15601 [09:15<00:00, 28.09it/s]


In [6]:
addrs = np.sort(glob('data/v2/files/high_res/0.0/train/*.hdf5'))

In [10]:
addrs[0][:30]

'data/v2/files/high_res/0.0/tra'

['directions', 'distances', 'has_sharp', 'has_smell_bad_face_sampling', 'has_smell_coarse_surfaces_by_angles', 'has_smell_coarse_surfaces_by_num_faces', 'has_smell_deviating_resolution', 'has_smell_mismatching_surface_annotation', 'has_smell_sharpness_discontinuities', 'item_id', 'normals', 'num_sharp_curves', 'num_surfaces', 'orig_face_indexes', 'orig_vert_indices', 'points', 'voronoi']


In [17]:
i=0
n_10 = np.load('data/v2/files/high_res/0.0/train/10_train_'+str(i)+'.npy', allow_pickle=True)
n_100 = np.load('data/v2/files/high_res/0.0/train/100_train_'+str(i)+'.npy', allow_pickle=True)
with h5py.File('data/v2/files/high_res/0.0/test/train_'+str(i)+'.hdf5', 'a') as f:
    f.create_dataset("normals_estimation_10", data=n_10)
    f.create_dataset("normals_estimation_100", data=n_100)

TypeError: Object dtype dtype('O') has no native HDF5 equivalent

In [2]:
i=10
n_10 = np.load('data/v2/files/high_res/0.0/train/100_train_'+str(i)+'.npy', allow_pickle=True)
for i in range(len(n_10)):
    if n_10[i].shape!=(4096, 3):
        print(i)
#     break

FileNotFoundError: [Errno 2] No such file or directory: 'data/v2/files/high_res/0.0/train/100_train_10.npy'

In [45]:
np.loadtxt('data/v2/files/high_res/0.0/train/xyz_splitted/train_1_points_102.xyz').shape

(4096, 6)

In [60]:
np.save('data/v2/files/high_res/0.0/val/train_0', ns)

In [62]:
np.array(ns)[0,100]

array([ 0.,  1., -0.])

In [10]:
for i in range(11):
    addrs = np.sort(glob('data/v2/files/high_res/0.0/train/xyz_splitted_100/train_'+str(i)+'_*.xyz'))
    print(len(addrs))

16384
16384
16384
16384
16384
16384
16384
16384
16384
16384
16384


In [ ]:
for i in range(0,11):
    addrs = np.sort(glob('data/v2/files/high_res/0.0/train/xyz_splitted/train_'+str(i)+'_*.xyz'))
    ns = []
    for j in tqdm(range(len(addrs))):
        n = []
        with open('data/v2/files/high_res/0.0/train/xyz_splitted/train_'+str(i)+'_points_'+str(j)+'.xyz', 'r') as f:
            for line in f.readlines():
                n.append(line.split(' ')[-4:-1])
        ns.append(n)
    np.save('data/v2/files/high_res/0.0/train/10_train_'+str(i), ns)

100%|██████████| 16384/16384 [05:31<00:00, 49.38it/s]


In [21]:
with open('data/v2/files/high_res/0.0/train/xyz_splitted/train_'+str(i)+'_points_'+str(j)+'.xyz') as f:
    for line in f.readlines():
        print(line.split(' ')[-4:-1])
        break

TypeError: float() argument must be a string or a number, not 'list'

In [13]:
pd.read_csv('data/v2/files/high_res/0.0/train/xyz_splitted/train_0_points_13.xyz', sep=' ',
            header=None).shape

(4096, 7)

In [13]:
for i in range(11):
    addrs = np.sort(glob('data/v2/files/high_res/0.0/train/xyz_splitted_100/train_'+str(i)+'_*.xyz'))
    ns = []
    for j in tqdm(range(len(addrs))):
        ns.append(np.loadtxt('data/v2/files/high_res/0.0/train/xyz_splitted_100/train_'+str(i)+'_points_'+str(j)+'.xyz')[:,3:])
    np.save('data/v2/files/high_res/0.0/train/100_train_'+str(i), ns)

100%|██████████| 16384/16384 [10:19<00:00, 26.46it/s]


In [32]:
sel = dist.sum(-1) < 4096

In [33]:
data = data[sel]
label = label[sel]

In [34]:
import k3d
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm

def get_colors(pred, cmap):
    hex_colors = []
    norm = mpl.colors.Normalize(vmin=-1, vmax=0)
    cmap = cmap
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    for i in pred:
#         hex_colors.append(m.to_rgba(i))
        hex_colors.append(int(mpl.colors.to_hex(m.to_rgba(i)).replace('#','0x'),16))
    hex_colors = np.array(hex_colors, 'uint32')
    return(hex_colors)

# dataset[43072]


# data = dataset[43072]['points'].numpy()

# lab = dataset[43072]['distances'].numpy()

for sample in range(10):

    p = k3d.plot()


    col_tr = get_colors(-label[sample], cm.coolwarm)

    points = k3d.points(data[sample], col_tr, point_size=0.05, shader='mesh')

    p += points

    f = open(str(sample)+".html", 'w')
    f.write(p.get_snapshot())
    f.close()

/trinity/home/a.matveev/.local/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [6]:
with h5py.File('tmp/test/train_1.hdf5', 'r') as f:
    v = f['voronoi'][:]

with h5py.File('tmp/test_1/train_1.hdf5', 'a') as f:
    f.create_dataset("voronoi", data=v)
    
with h5py.File('tmp/test_1/train_1.hdf5', 'a') as f:
    print(list(f.keys()))

['camera_pose', 'directions', 'distances', 'has_sharp', 'has_smell_bad_face_sampling', 'has_smell_coarse_surfaces_by_angles', 'has_smell_coarse_surfaces_by_num_faces', 'has_smell_depth_discontinuity', 'has_smell_deviating_resolution', 'has_smell_mesh_self_intersections', 'has_smell_mismatching_surface_annotation', 'has_smell_raycasting_background', 'has_smell_sharpness_discontinuities', 'image', 'item_id', 'mesh_scale', 'normals', 'num_sharp_curves', 'num_surfaces', 'orig_face_indexes', 'orig_vert_indices', 'voronoi']
